# Instructions

The assignment consists of three tasks:

- Run the T-test for the means of two independent samples underlying the statement "IRE binding activity was significantly reduced in failing hearts" (originally published by Haddad et al. in https://doi.org/10.1093/eurheartj/ehw333) using the following example data.

| non-failing heart (NF) | failing heart (F) |
| ---------------------- | ----------------- |
| 95 | 50 |
| 103 | 35 |
| 99 | 21 | 
| &nbsp; | 15 | 
| &nbsp; | 7 | 
| &nbsp; | 40 |

- Describe the statistical hypothesis test in machine readable form following the [statistical methods ontology concept for "two sample t-test with unequal variance"](https://www.ebi.ac.uk/ols/ontologies/stato/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FSTATO_0000304) using Semantic Web technologies, namely the Resource Description Framework (RDF)
- Process the resulting machine readable description using Semantic Web technologies, namely the SPARQL Protocol and RDF Query Language.

Please return the assignment with all outputs visible (i.e., do not clear the outputs).

Good luck!

In [1]:
!pip install rdflib pandas scipy numpy

In [8]:
# Import all required libraries (some are missing)
import pandas as pd
import numpy as np
from rdflib import Graph, URIRef, BNode, Literal
from rdflib.namespace import RDF, XSD
from scipy import stats

In [9]:
# Run the T-test for the means of two independent samples using the example data

labels = ['non-failing heart (NF)', 'failing heart (F)']
data = [(95,50), (103,35), (99,21), (np.nan, 15), (np.nan, 7), (np.nan, 40)]

# Creating a dataframe from data and labels
df = pd.DataFrame.from_records(data, columns=labels)

ttest = stats.ttest_ind(df['non-failing heart (NF)'],
              df['failing heart (F)'],
              equal_var=False, nan_policy='omit')
# Print the p-value
ttest.pvalue

5.3187252634905425e-05

In [14]:
# Describe the statistical hypothesis test in machine readable form

# First, we initialize an RDF Graph and bind some prefixes
g = Graph()
g.bind('obo', 'http://purl.obolibrary.org/obo/')
g.bind('ex', 'http://example.org/')

# Next, we define some needed vocabulary
obo = dict()
obo['two sample t-test with unequal variance'] = URIRef('http://purl.obolibrary.org/obo/STATO_0000304')
obo['has specified input'] = URIRef('http://purl.obolibrary.org/obo/OBI_0000293')
obo['has specified output'] = URIRef('http://purl.obolibrary.org/obo/OBI_0000299')
obo['p-value'] = URIRef('http://purl.obolibrary.org/obo/OBI_0000175')
obo['has value specification'] = URIRef('http://purl.obolibrary.org/obo/OBI_0001938')
obo['scalar value specification'] = URIRef('http://purl.obolibrary.org/obo/OBI_0001931')
obo['has specified numeric value'] = URIRef('http://purl.obolibrary.org/obo/OBI_0001937')
obo['iron-responsive element binding'] = URIRef('http://purl.obolibrary.org/obo/GO_0030350')

# Now, populate the graph with statements
# As an example, the following statement types the resource (blank node) 'n1' as 'two sample t-test with unequal variance'
n1 = BNode()
g.add((n1, RDF.type, obo['two sample t-test with unequal variance']))

# Now, state that the t-test has iron-responsive element binding as a specified input (one statement to complete)
n2 = BNode()
g.add((n1, obo['has specified input'], n2))
g.add((n2, RDF.type, obo['iron-responsive element binding']))

# Next, state that the t-test has the p-value computed above as a specified output (two statements to complete)
n3 = BNode()
g.add((n1, obo['has specified output'], n3))
g.add((n3, RDF.type, obo['p-value']))

# The literal value of the p-value is represented as a scalar value specification with a specified numeric value (three statements to complete)
n4 = BNode()
n5 = BNode()
g.add((n3, obo['has value specification'], n4))
g.add((n4, RDF.type, obo['scalar value specification']))
g.add((n4, obo['scalar value specification'], n5))
# Type the literal as double
g.add((n4, obo['has specified numeric value'], Literal(ttest.pvalue, datatype=XSD.double)))

# Finally, serialize and print the graph in RDF/XML format
print(g.serialize(format='xml'))

b'<?xml version="1.0" encoding="UTF-8"?>\n<rdf:RDF\n   xmlns:obo="http://purl.obolibrary.org/obo/"\n   xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"\n>\n  <rdf:Description rdf:nodeID="N33a8c882b2fa4d9d8f6164ba76d15b51">\n    <rdf:type rdf:resource="http://purl.obolibrary.org/obo/STATO_0000304"/>\n    <obo:OBI_0000299 rdf:nodeID="Nee9b6c2846014cfebd437f9d857dd8d0"/>\n    <obo:OBI_0000293 rdf:nodeID="Nb04df16dc254493794ac09c7be30510d"/>\n  </rdf:Description>\n  <rdf:Description rdf:nodeID="N4276298f8f254bf9800a161168c8ee25">\n    <obo:OBI_0001937 rdf:datatype="http://www.w3.org/2001/XMLSchema#double">5.31872526349e-05</obo:OBI_0001937>\n    <rdf:type rdf:resource="http://purl.obolibrary.org/obo/OBI_0001931"/>\n    <obo:OBI_0001931 rdf:nodeID="N406025526c0e48b6a959996f51dfb703"/>\n  </rdf:Description>\n  <rdf:Description rdf:nodeID="Nee9b6c2846014cfebd437f9d857dd8d0">\n    <obo:OBI_0001938 rdf:nodeID="N4276298f8f254bf9800a161168c8ee25"/>\n    <rdf:type rdf:resource="http://purl.

In [12]:
# Process the machine readable statistical hypothesis test by completing the following SPARQL query that returns the p-value

q = """
PREFIX obo: <http://purl.obolibrary.org/obo/>

SELECT ?pvalue WHERE {
  ?r a obo:STATO_0000304 .                  #two sample t-test with unequal variance
  ?r obo:OBI_0000293 [ a obo:GO_0030350 ] . #has specified input of an iron-responsive element binding
  ?r obo:OBI_0000299 [                      #also has a specified output
       a obo:OBI_0000175 ;                  #of p-value
       obo:OBI_0001938 [                    #that has value specification
           a obo:OBI_0001931 ;              #of a scalar value specification
           obo:OBI_0001937 ?pvalue          #and has specified numeric value... output ?pvalue
       ]
   ]
}
"""

for qs in g.query(q):
    print(qs[0])

5.31872526349e-05
